<a href="https://colab.research.google.com/github/alirustama/Data-Analyst/blob/master/DataExtraction_%26_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:

import pandas as pd
import requests
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
df = pd.read_excel('/content/cik_list.xlsx')
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [37]:

l = 'https://www.sec.gov/Archives/'
links = [l+x for x in df['SECFNAME']]
print(links)

['https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002278.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002401.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002402.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950172-99-000362.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950170-99-000775.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950172-99-000584.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950170-99-001005.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000950172-99-001074.txt', 'https://www.sec.gov/Archives/edgar/data/3662/0000

In [38]:

reports = []
for url in links:
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    reports.append(soup.get_text())

print(f' {len(reports)} reports saved')

 152 reports saved


In [39]:
with open('/content/StopWords_Generic.txt','r') as f:
    stop_words = f.read()

stop_words = stop_words.split('\n')
print(f' Stop Words are {len(stop_words)}')

 Stop Words are 121


In [40]:

master_dict = pd.read_excel('/content/LoughranMcDonald_MasterDictionary_2020.xlsx')
master_dict.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Complexity,Syllables,Source
0,AARDVARK,1,312,1.422050e-08,1.335201e-08,3.700747e-06,96,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.367356e-10,8.882163e-12,9.362849e-09,1,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,4.102067e-10,1.200533e-10,5.359747e-08,7,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,15,6.836779e-10,4.080549e-10,1.406914e-07,14,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8009,3.650384e-07,3.798698e-07,3.523914e-05,1058,0,0,0,0,0,0,0,0,3,12of12inf


In [41]:
positive_dictionary = [x for x in master_dict[master_dict['Positive'] != 0]['Word']]

negative_dictionary = [x for x in master_dict[master_dict['Negative'] != 0]['Word']]

print(f"Total positve words in dictionary are {len(positive_dictionary)}")
print(f"Total negative words in dictionary are {len(negative_dictionary)}")

Total positve words in dictionary are 354
Total negative words in dictionary are 2355


In [42]:

uncertainity = pd.read_excel('uncertainty_dictionary.xlsx')
uncertainity_words = list(uncertainity['Word'])

constraining = pd.read_excel('constraining_dictionary.xlsx')
constraining_words = list(constraining['Word'])

In [43]:
def tokenize(text):
    text = re.sub(r'[^A-Za-z]',' ',text.upper())
    tokenized_words = word_tokenize(text)
    return tokenized_words

def remove_stopwords(words, stop_words):
    return [x for x in words if x not in stop_words]
    
def countfunc(store, words):
    score = 0
    for x in words:
        if(x in store):
            score = score+1
    return score

def sentiment(score):
    if(score < -0.5):
        return 'Most Negative'
    elif(score >= -0.5 and score < 0):
        return 'Negative'
    elif(score == 0):
        return 'Neutral'
    elif(score > 0 and score < 0.5):
        return 'Positive'
    else:
        return 'Very Positive'

In [44]:
def polarity(positive_score, negative_score):
    return (positive_score - negative_score)/((positive_score + negative_score)+ 0.000001)
     

def subjectivity(positive_score, negative_score, num_words):
    return (positive_score+negative_score)/(num_words+ 0.000001)

def syllable_morethan2(word):
    if(len(word) > 2 and (word[-2:] == 'es' or word[-2:] == 'ed')):
        return False
    
    count =0
    vowels = ['a','e','i','o','u']
    for i in word:
        if(i.lower() in vowels):
            count = count +1
        
    if(count > 2):
        return True
    else:
        return False
    
def fog_index_cal(average_sentence_length, percentage_complexwords):
    return 0.4*(average_sentence_length + percentage_complexwords)

In [45]:

sections = ["Management's Discussion and Analysis",
            "Quantitative and Qualitative Disclosures about Market Risk\n",
            "Risk Factors\n"]
caps = [x.upper() for x in sections]

caps.extend(sections)

In [61]:
col = ['mda','qqdmr','rf']
var = ['positive_score',
      'negative_score',
      'polarity_score',
      'average_sentence_length',
      'percentage_of_complex_words',
      'fog_index',
      'complex_word_count',
      'word_count',
      'uncertainity_score',
      'constraining_score',
      'positive_word_proportion',
      'negative_word_proportion',
      'uncertainity_word_proportion',
      'constraining_word_proportion',
      'constraining_words_whole_report']


for c in col:
    for v in var[:-1]:
        df[c+'_'+v] = 0.0

df[var[-1]] = 0.0

In [62]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainity_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainity_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainity_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainity_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainity_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainity_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:

section_map = {i:j for i,j in zip(sections, col)}
s_map = {i.upper():j for i,j in zip(sections, col)}

section_map.update(s_map)
print(section_map)

{"Management's Discussion and Analysis": 'mda', 'Quantitative and Qualitative Disclosures about Market Risk\n': 'qqdmr', 'Risk Factors\n': 'rf', "MANAGEMENT'S DISCUSSION AND ANALYSIS": 'mda', 'QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK\n': 'qqdmr', 'RISK FACTORS\n': 'rf'}


In [64]:
for i in range(len(reports)):
  text = re.sub('Item','ITEM',reports[i])
  for j in caps:
    x = re.search('ITEM\s+[\d]\(*[A-Za-z]*\)*.*\s+\-*\s*'+j, text)
        
    if x:
      start,end = x.span()
      content = (text[start:]).split('ITEM')[1]
      if ('...' not in content) and ('. . .' not in content) and len(content) > 200:
        tokenized_words = tokenize(content)
                #print(f'Total tokenized words are {len(tokenized_words)}')
        words = remove_stopwords(tokenized_words, stop_words)
        num_words = len(words)
                #print(f'Total words after removing stop words are {len(words)}')
        positive_score = countfunc(positive_dictionary, words)
        negative_score = countfunc(negative_dictionary, words)
                #print(f'Total positive score is {positive_score}')
                #print(f'Total negative score is {negative_score}')
        polarity_score = polarity(positive_score, negative_score)
                #print(polarity_score)
        subjectivity_score = subjectivity(positive_score, negative_score, num_words)
                #print(subjectivity_score)
                #print(sentiment(polarity_score))
                
        sentences = sent_tokenize(content)
        num_sentences = len(sentences)
        average_sentence_length = num_words/num_sentences
                #print(average_sentence_length)
                
                
        num_complexword =0
        uncertainity_score = 0
        constraining_score = 0
        for word in words:
          if(syllable_morethan2(word)):
            num_complexword = num_complexword+1
                        
          if(word in uncertainity_words):
            uncertainity_score = uncertainity_score+1
                        
          if(word in constraining_words):
            constraining_score = constraining_score+1
                        
                #print(num_complexword)
                #print(uncertainity_score)
                #print(constraining_score)
                
                
        percentage_complexwords = num_complexword/num_words
                #print(percentage_complexwords)
        fog_index = fog_index_cal(average_sentence_length, percentage_complexwords)
                #print(fog_index)
                
        positive_word_proportion = positive_score/num_words
        negative_word_proportion = negative_score/num_words
        uncertainity_word_proportion = uncertainity_score/num_words
        constraining_word_proportion = constraining_score/num_words
        df.at[i,section_map[j]+'_positive_score'] = positive_score
        df.at[i,section_map[j]+'_negative_score'] = negative_score
        df.at[i,section_map[j]+'_polarity_score'] = polarity_score
        df.at[i,section_map[j]+'_average_sentence_length'] = average_sentence_length
        df.at[i,section_map[j]+'_percentage_of_complex_words'] = percentage_complexwords
        df.at[i,section_map[j]+'_fog_index'] = fog_index
        df.at[i,section_map[j]+'_complex_word_count'] = num_complexword
        df.at[i,section_map[j]+'_word_count'] = num_words
        df.at[i,section_map[j]+'_uncertainity_score'] = uncertainity_score
        df.at[i,section_map[j]+'_constraining_score'] = constraining_score
        df.at[i,section_map[j]+'_positive_word_proportion'] = positive_word_proportion
        df.at[i,section_map[j]+'_negative_word_proportion'] = negative_word_proportion
        df.at[i,section_map[j]+'_uncertainity_word_proportion'] = uncertainity_word_proportion
        df.at[i,section_map[j]+'_constraining_word_proportion'] = constraining_word_proportion
  constraining_words_whole_report = 0
  tokenized_report_words = tokenize(reports[i])
  report_words = remove_stopwords(tokenized_report_words, stop_words)
  for word in report_words:
    if word in constraining_words:
      constraining_words_whole_report = 1+ constraining_words_whole_report
    #print(constraining_words_whole_report)
    df.at[i,'constraining_words_whole_report'] = constraining_words_whole_report
              

In [68]:


df.to_excel('/content/Output Data Structure.xlsx')

